# Tracing with Function Calls and Chains

In [ ]:
import json
import os
from typing import List, Dict, Any, Union

from parea import Parea, trace, trace_insert
from parea.schemas import LLMInputs, Message, ModelParams, Completion

p = Parea(api_key=os.environ["PAREA_API_KEY"])

In [5]:
LIMIT = 2  # limit any loops to 2 iterations for demo purposes

COURSE_FUNCTIONS = [
    {
        "name": "generate_course_outline",
        "description": "Generates a course outline",
        "parameters": {
            "type": "object",
            "properties": {
                "Description": {"type": "string", "description": "The description of the course"},
                "chapters": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string", "description": "The title of the chapter"},
                            "description": {"type": "string", "description": "The summary of the chapter"},
                        },
                        "required": ["name", "description"],
                    },
                    "description": "The chapters included in the course",
                    "minItems": 1,
                    "maxItems": LIMIT,
                },
            },
            "required": ["Description", "chapters"],
        },
    }
]

CHAPTER_FUNCTIONS = [
    {
        "name": "generate_chapter_outline",
        "description": "Generates a chapter outline",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "The title of the chapter"},
                "description": {"type": "string", "description": "The summary of the chapter"},
                "sections": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string", "description": "The title of the section"},
                            "description": {"type": "string", "description": "The summary of the section"},
                        },
                        "required": ["name", "description"],
                    },
                    "description": "The sections included in the chapter",
                    "minItems": 1,
                    "maxItems": LIMIT,
                },
            },
            "required": ["name", "description", "sections"],
        },
    }
]

In [ ]:
USER = "alice@bob.com"


# Create a reusable call lmm helper function
# Parea SDK is automatically traced
def call_llm(messages: List[Dict[str, str]], name: str = "LLM Call") -> str:
    return p.completion(
        Completion(
            llm_configuration=LLMInputs(
                model="gpt-3.5-turbo-1106",
                model_params=ModelParams(temp=0.0, max_length=512),
                messages=[Message(**m) for m in messages],
                functions=COURSE_FUNCTIONS + CHAPTER_FUNCTIONS,
            ),
            end_user_identifier=USER,
            trace_name=name,
        )
    ).content


# Helper function to get function call arguments if they exists
def get_function_call_or_content(response: str) -> Union[str, Dict[str, Any]]:
    # Function calls are returned in triple backticks
    parsed_response = response.replace("```", "")
    # return function call or content string
    try:
        return json.loads(parsed_response).get("arguments")
    except Exception as e:
        return parsed_response

In [ ]:
# PROMPTS


def get_course(topic: str):
    _course_outline = call_llm(
        messages=[{"role": "user", "content": f"Generate a course outline on {topic}"}],
        name=f"Create {topic} Course Outline",
    )
    course_outline = get_function_call_or_content(_course_outline)
    print(json.dumps(course_outline, indent=4))
    return course_outline


# I want to group all llm calls into one trace,
# so I add the trace decorator to create a parent trace,
# each call_llm will be a child span
@trace(name="Get Chapters")
def get_chapters(course_outline: Dict[str, str]):
    chapter_outlines = course_outline.get("chapters", [])
    chapters = [
        get_function_call_or_content(
            call_llm(
                messages=[{"role": "user", "content": f"Generate a chapter outline on {chapter.get('name')}, with description {chapter.get('description')}"}],
                name=f"Create Chapter {idx}",
            )
        )
        for idx, chapter in enumerate(chapter_outlines, start=1)
    ]
    return chapters[:LIMIT]


@trace
def get_sections(chapter: Dict[str, str]):
    chapter_name = chapter.get("name", "section")
    # I want the trace name to be dynamic based on the chapter param,
    # so I can use Parea trace_insert helper method to add dynamic data
    trace_insert({"trace_name": f"Get {chapter_name} Sections"})

    section_outlines = chapter.get("sections", [])
    sections = [
        get_function_call_or_content(
            call_llm(
                messages=[{"role": "user", "content": f"""Generate a section outline on {section['name']}, with description {section['description']}"""}],
                name=f"Create Section {idx}",
            )
        )
        for idx, section in enumerate(section_outlines, start=1)
    ]
    return sections[:LIMIT]

In [ ]:
@trace(name="Get Sections")
def get_all_sections(chapters: List[Dict[str, str]]):
    return [get_sections(chapter) for chapter in chapters]


@trace
def run_creation(topic: str):
    trace_insert({"trace_name": f"Course on {topic}"})
    course_outline = get_course(topic)
    chapters = get_chapters(course_outline)
    sections = get_all_sections(chapters)
    return sections


@trace(name="Generate Courses for Topics")
def main(topics):
    for topic in topics:
        print(f"\n New Topic: {topic} \n")
        run_creation(topic)

In [ ]:
main(["Artificial Intelligence", "Machine Learning", "Deep Learning"])

### Trace Log View in UI

![TraceView](img/trace_log_view.png)